# Task 5D

| | |
|---|---|
| Name | Bandi Krishna Chaitanya |
| Deakin ID | s225170881 |
| Submission Date | |

## Introduction

Data from https://www.kaggle.com/datasets/mssmartypants/paris-housing-price-prediction is used for this task. The dataset contains information about various properties in Paris, including their prices, locations, and other features. The goal is to predict the price of a property based on its features.

## Imports

In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold  
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance
import shap
import joblib

## Data Exploration

### Basic Information about the Dataset

In [ ]:
# load the dataset
paris_real_estate_df = pd.read_csv('ParisHousing.csv')

In [ ]:
# shape of the dataset
print(f"Shape of the dataset: {paris_real_estate_df.shape}")

In [ ]:
# print all features and their data types using tabulate
print(tabulate(paris_real_estate_df.dtypes.reset_index(), headers=['Feature', 'Data Type'], tablefmt='grid'))

All the 17 features in the dataset are numerical. The dataset contains 10,000 rows and 17 columns. 

There may be some features that are recognized as numerical but are actually categorical. For example, as can be seen from the list of features above, `hasYard`, `hasPool` etc., are binary features that indicate whether a property has a yard, pool, etc. These features should be treated as categorical features rather than numerical features.

We shall explore the dataset to identify such features and treat them accordingly.

Target feature is `price` and the rest are input features.

### Check for Missing Values

In [ ]:
# print the number of missing values in each feature using tabulate
missing_values = paris_real_estate_df.isnull().sum().reset_index()
missing_values.columns = ['Feature', 'Missing Values']
print(tabulate(missing_values, headers='keys', tablefmt='grid'))    

The dataset does not contain any missing values.

### Univariate Analysis

In [ ]:
# plot the histogram kde plot for each feature using seaborn as a grid
plt.figure(figsize=(20, 20))
for i, column in enumerate(paris_real_estate_df.columns):
    plt.subplot(5, 4, i + 1)
    sns.histplot(paris_real_estate_df[column], kde=True)
    plt.title(column)
plt.tight_layout()
plt.show()

From the histogram plot above, we can see that the following features are binary in nature and should be treated as categorical features:

- `hasYard`
- `hasPool`
- `isNewBuilt`
- `hasStormProtector`
- `hasStorageRoom`

And there are some features that appear to be categorical. We shall inspect the number of unique values and their counts to confirm this.

- `cityPartRange`
- `numPrevOwners`
- `hasGuestRoom`

In [ ]:
# inspect the unique values and their counts for potential categorical features
categorical_features = ['cityPartRange', 'numPrevOwners', 'hasGuestRoom']
for feature in categorical_features:
    unique_counts = paris_real_estate_df[feature].value_counts().reset_index()
    unique_counts.columns = ['Value', 'Count']
    print(f"\nUnique values and counts for {feature}:")
    unique_counts = unique_counts.sort_values(by='Value')
    print(tabulate(unique_counts, headers='keys', tablefmt='grid'))

All these three features appear to be categorical in nature as they have a limited number of unique values. We shall treat them as categorical features.

These are already labeled and do not require any further processing.

In [ ]:
# describe the dataset
paris_real_estate_description = paris_real_estate_df.describe().transpose()

paris_real_estate_description

### Checking for outliers

In [ ]:
# plot boxplots for all features to identify outliers
plt.figure(figsize=(20, 24))
for i, column in enumerate(paris_real_estate_df.columns):
    plt.subplot(6, 4, i + 1)
    sns.boxplot(x=paris_real_estate_df[column])
    plt.title(column)
plt.tight_layout()
plt.show()

There don't seem to be any outliers in the dataset, as can be seen from the boxplot above. The features are already in a suitable format for modeling.

In [ ]:
# number of unique values in cityCode
unique_city_codes = paris_real_estate_df['cityCode'].nunique()
print(f"\nNumber of unique city codes: {unique_city_codes}")

Based on the descriptive statistics of the dataset, and the histogram plots, we can summarize the features as follows:

- There are 17 features in total.
- The target feature is `price`.
- There are no missing values in the dataset.
- Based on the box plots, there are no significant outliers in the dataset.
- Some of the features are binary and should be treated as categorical features:
  - `hasYard`
  - `hasPool`
  - `isNewBuilt`
  - `hasStormProtector`
  - `hasStorageRoom`
- Some features appear to be categorical based on the number of unique values:
  - `cityPartRange`
  - `numPrevOwners`
  - `hasGuestRoom` 
- `cityCode` may not be necessary for the model, as we can use `cityPartRange` to represent the city part.
- `made` must be treated as a categorical feature, as it represents the year the property was built and has a limited number of unique values.
- `price` does not vary much based on the year the property was built, as per the plots made per city part over the years.

### Price Analysis over the Years for each City Part

In [ ]:
# for a given cityPartRange, plot the price vs yearBuilt in a grid of plots
city_part_ranges = paris_real_estate_df['cityPartRange'].unique()
plt.figure(figsize=(20, 20))
for i, city_part in enumerate(city_part_ranges):
    plt.subplot(5, 4, i + 1)
    subset = paris_real_estate_df[paris_real_estate_df['cityPartRange'] == city_part]
    sns.scatterplot(x=subset['made'], y=subset['price'], alpha=0.5)
    plt.title(f'City Part: {city_part}')
    plt.xlabel('Year Built')
    plt.ylabel('Price')
plt.tight_layout()
plt.show()

Based on the plots above, it looks like the prices of the houses don't vary much based on the year they were built. They are more or less uniformly distributed across the years.

### Summary of Data Exploration

## Data Preprocessing

### Dropping and Changing Feature Types

In [ ]:
# Drop the 'cityCode' feature as it has too many unique values
paris_real_estate_processing = paris_real_estate_df.drop(columns=['cityCode'])

In [ ]:
# binary features that should be treated as categorical
binary_features = ['hasYard', 'hasPool', 'isNewBuilt', 'hasStormProtector', 'hasStorageRoom']
for feature in binary_features:
    paris_real_estate_processing[feature] = paris_real_estate_processing[feature].astype('category')

In [ ]:
# other categorical features
categorical_features = ['cityPartRange', 'numPrevOwners', 'hasGuestRoom']
for feature in categorical_features:
    paris_real_estate_processing[feature] = paris_real_estate_processing[feature].astype('category')

Till now, we did the following preprocessing steps:

- Dropped the `cityCode` feature as it has too many unique values.
- Converted the binary features to categorical features
- Converted the already identified categorical features to categorical data type. These are already labeled and do not require any further processing.

Let us now encode the `made` and `cityPartRange` features as categorical features.

For `made`, since it represents the year the property was built, simply converting it to a categorical type is sufficient. This will allow the model to treat it as a categorical feature without needing further encoding.

This is because years have natural ordering and meaningful distance criteria already.

For `cityPartRange`, we can use one-hot encoding to represent the different city parts.

### Correlation Analysis

In [ ]:
# check correlation matrix
plt.figure(figsize=(20, 20))
correlation_matrix = paris_real_estate_processing.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Correlation Matrix')
plt.show()  

There is a PERFECT correlation between `price` and `squareMeters`, which is strange in the real world, but it is what it is in this dataset. This means that the price of a property is directly proportional to its size in square meters.

There doesn't seem to be any heavy correlation between the input features, so we can proceed with the preprocessing steps without worrying about multicollinearity.

### Splitting the Dataset for Training and Testing

In [ ]:
# save the processed DataFrame to a CSV file
paris_real_estate_processing.to_csv('paris_real_estate_processed.csv', index=False)

In [ ]:
# train and test split
X = paris_real_estate_processing.drop(columns=['price'])
y = paris_real_estate_processing['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### One-Hot Encoding of Categorical Feature - `cityPartRange`

In [ ]:
# categorical encoding for 'made' and one-hot encoding for 'cityPartRange' for training and testing sets
X_train = pd.get_dummies(X_train, columns=['cityPartRange'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['cityPartRange'], drop_first=True)

# change the feature 'made' to categorical for both training and testing sets
X_train['made'] = X_train['made'].astype('category')
X_test['made'] = X_test['made'].astype('category')

In [ ]:
# print the train and test sets structure
print("\nTraining set structure:")
print(tabulate(X_train.dtypes.reset_index(), headers=['Feature', 'Data Type'], tablefmt='grid'))
print("\nTesting set structure:")
print(tabulate(X_test.dtypes.reset_index(), headers=['Feature', 'Data Type'], tablefmt='grid')) 

In [ ]:
# shape of the train and test sets
print(f"\nShape of the training set: {X_train.shape}")
print(f"Shape of the testing set: {X_test.shape}")

### Standardization of Numerical Features

Below features of the dataset need to be standardized:

- squareMeters
- numberOfRooms
- floors
- basement
- attic
- garage

In [ ]:
# standardize the features that need to be standardized
features_to_standardize = ['squareMeters', 'numberOfRooms', 'floors', 'basement', 'attic', 'garage']
scaler = StandardScaler()
X_train[features_to_standardize] = scaler.fit_transform(X_train[features_to_standardize])
X_test[features_to_standardize] = scaler.transform(X_test[features_to_standardize])

In [ ]:
# write the processed training and testing sets to CSV files
X_train.to_csv('X_train_processed.csv', index=False)
X_test.to_csv('X_test_processed.csv', index=False)
y_train.to_csv('y_train_processed.csv', index=False)
y_test.to_csv('y_test_processed.csv', index=False)

## Model Building

Since the target feature `price` is a continuous variable, we will use regression models to predict the price of the properties. 

Based on the instructions, the following metrics will be used to evaluate the models:

- Mean Absolute Error (MAE)
- Root Mean Squared Error (RMSE)
- R-squared (R2)

We shall try the following regression models:

- Linear Regression
- Random Forest Regressor  
- Gradient Boosting Regressor

We shall use K-Fold Cross-Validation to evaluate the models. This will help us to get a better estimate of the model's performance on unseen data.

### Linear Regression with K-Fold Cross-Validation

In [ ]:
# kf object with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# build a linear regression model with default parameters
linear_model = LinearRegression()

# do cross-validation
linear_scores = cross_val_score(linear_model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

# get MSE scores
linear_mse_scores = -linear_scores  # negate the scores to get positive MSE values

# get RMSE scores
linear_rmse_scores = linear_mse_scores ** 0.5

# get R^2 scores
linear_r2_scores = cross_val_score(linear_model, X_train, y_train, cv=kf, scoring='r2')

# print all three scores using tabulate
linear_scores_table = pd.DataFrame({
    'Fold': range(1, len(linear_mse_scores) + 1),
    'MSE': linear_mse_scores,
    'RMSE': linear_rmse_scores,
    'R^2': linear_r2_scores
})
print("\nLinear Regression Model Cross-Validation Scores:")
print(tabulate(linear_scores_table, headers='keys', tablefmt='grid'))

As seen above, the R-squared value for the Linear Regression model is 1.0, which indicates that the model is able to explain 100% of the variance in the target variable `price`. 

This is unusual and generally indicates that the model is overfitting the training data. In real-world scenarios, we would expect the R-squared value to be less than 1.0, indicating that the model is not able to explain all the variance in the target variable.

However, since the squareMeters feature is perfectly correlated with the price, the model is probably been able to explain all the variance in the target variable.

**Summary of K-Fold Cross-Validation for Linear Regression:**

Based on the 5 folds of cross-validation, we see an uniform performance across all the folds.

### Random Forest Regressor with K-Fold Cross-Validation

In [ ]:
# random forest regressor with default parameters
rf_model = RandomForestRegressor(random_state=42)

# do cross-validation
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

# get MSE scores
rf_mse_scores = -rf_scores  # negate the scores to get positive MSE values

# get RMSE scores
rf_rmse_scores = rf_mse_scores ** 0.5

# get R^2 scores
rf_r2_scores = cross_val_score(rf_model, X_train, y_train, cv=kf, scoring='r2')

# print all three scores using tabulate
rf_scores_table = pd.DataFrame({
    'Fold': range(1, len(rf_mse_scores) + 1),
    'MSE': rf_mse_scores,
    'RMSE': rf_rmse_scores,
    'R^2': rf_r2_scores
})
print("\nRandom Forest Regressor Cross-Validation Scores:")
print(tabulate(rf_scores_table, headers='keys', tablefmt='grid'))

Based on the 5 folds of cross-validation, here are the observations:

- The performance of the Random Forest Regressor is consistent across all folds.
- We still see a near-perfect R-squared value of 1.0, indicating that the model is able to explain 100% of the variance in the target variable `price`.
- Both MSE and RMSE values are higher than those of the Linear Regression model, indicating that the Random Forest Regressor is not performing as well as the Linear Regression model in this case.
- This lower performance of Random Forest Regressor indicates that there is a much better linear relationship between the input features and the target feature `price`, which is why the Linear Regression model is performing better. And this is expected as the `squareMeters` feature is perfectly correlated with the `price` feature, which is a linear relationship.

In [ ]:
# gradient boosting regressor with default parameters
gb_model = GradientBoostingRegressor(random_state=42)

# do cross-validation
gb_scores = cross_val_score(gb_model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

# get MSE scores
gb_mse_scores = -gb_scores  # negate the scores to get positive MSE values

# get RMSE scores
gb_rmse_scores = gb_mse_scores ** 0.5

# get R^2 scores
gb_r2_scores = cross_val_score(gb_model, X_train, y_train, cv=kf, scoring='r2')

# print all three scores using tabulate
gb_scores_table = pd.DataFrame({
    'Fold': range(1, len(gb_mse_scores) + 1),
    'MSE': gb_mse_scores,
    'RMSE': gb_rmse_scores,
    'R^2': gb_r2_scores
})
print("\nGradient Boosting Regressor Cross-Validation Scores:")
print(tabulate(gb_scores_table, headers='keys', tablefmt='grid'))

Here are the observations based on the K-Fold Cross-Validation for Gradient Boosting Regressor:

- The performance of the Gradient Boosting Regressor is consistent across all folds.
- We still see a near-perfect R-squared value of 1.0, indicating that the model is able to explain 100% of the variance in the target variable `price`.
- Both MSE and RMSE values are higher than those of the Linear Regression model as well as the Random Forest Regressor. GBR is probably overfitting the training data as well, but not as much as the Random Forest Regressor.

### Compare the three models with MSE, RMSE, and R-squared values

In [ ]:
# plot the mse across the folds for all three models
plt.figure(figsize=(12, 6))
plt.plot(linear_scores_table['Fold'], linear_scores_table['MSE'], marker='o', label='Linear Regression', color='blue')
plt.plot(rf_scores_table['Fold'], rf_scores_table['MSE'], marker='o', label='Random Forest Regressor', color='orange')
plt.plot(gb_scores_table['Fold'], gb_scores_table['MSE'], marker='o', label='Gradient Boosting Regressor', color='green')
plt.title('MSE Across Folds for Different Models')
plt.xlabel('Fold')
plt.ylabel('Mean Squared Error (MSE)')
plt.xticks(linear_scores_table['Fold'])
plt.legend()
plt.grid()
plt.show()

In [ ]:
# plot the rmse across the folds for all three models
plt.figure(figsize=(12, 6))
plt.plot(linear_scores_table['Fold'], linear_scores_table['RMSE'], marker='o', label='Linear Regression', color='blue')
plt.plot(rf_scores_table['Fold'], rf_scores_table['RMSE'], marker='o', label='Random Forest Regressor', color='orange')
plt.plot(gb_scores_table['Fold'], gb_scores_table['RMSE'], marker='o', label='Gradient Boosting Regressor', color='green')
plt.title('RMSE Across Folds for Different Models')
plt.xlabel('Fold')
plt.ylabel('Root Mean Squared Error (RMSE)')
plt.xticks(linear_scores_table['Fold'])
plt.legend()
plt.grid()
plt.show()

In [ ]:
# plot the R^2 across the folds for all three models
plt.figure(figsize=(12, 6))
plt.plot(linear_scores_table['Fold'], linear_scores_table['R^2'], marker='o', label='Linear Regression', color='blue')
plt.plot(rf_scores_table['Fold'], rf_scores_table['R^2'], marker='o', label='Random Forest Regressor', color='orange')
plt.plot(gb_scores_table['Fold'], gb_scores_table['R^2'], marker='o', label='Gradient Boosting Regressor', color='green')
plt.title('R^2 Across Folds for Different Models')
plt.xlabel('Fold')
plt.ylabel('R-squared')
# use log scale for y-axis
plt.yscale('log')
plt.xticks(linear_scores_table['Fold'])
plt.legend()
plt.grid()
plt.show()

| Model | Mean MSE | Mean RMSE | Mean R² | Performance Notes |
|-------|----------|-----------|---------|-------------------|
| Linear Regression | 3.61M | 1,897 | 1.000000 | Perfect fit due to linear relationship |
| Random Forest | 15.54M | 3,939 | 0.999998 | Near-perfect but adds complexity noise |
| Gradient Boosting | 424.77M | 20,610 | 0.999949 | Worst performance despite high R² |

### Key Insights in comparison of the three models:

**Performance Ranking (Best to Worst):**

1. **Linear Regression** - Perfect for this linear relationship
2. **Random Forest** - 4x higher MSE, slight imperfection 
3. **Gradient Boosting** - 118x higher MSE, significant overfitting

**Why This Happened:**

- All models achieve near-perfect R² due to the `squareMeters` perfect correlation
- **Linear Regression wins** because the true relationship is perfectly linear
- **Tree-based models (RF, GB) add unnecessary complexity** to a simple linear pattern
- **Gradient Boosting performs worst** - likely overfitting to training noise with aggressive boosting

**Conclusion:**

This demonstrates that **model complexity should match data complexity**. For perfectly linear relationships, simple linear regression is optimal. The tree-based models are solving the wrong problem - they're designed for non-linear patterns that don't exist in this dataset.

## Feature Importance Analysis

In [ ]:
# number of features in training set
num_features = X_train.shape[1]
print(f"\nNumber of features in training set: {num_features}")

We have 23 features in the training set. As seen earlier, the `squareMeters` feature dominates the dataset, and the model is able to explain 100% of the variance in the target variable `price`.

However, let's analyze the feature importance to understand the contribution of each feature in the model.

We shall use two methods to analyze feature importance:

1. Use model-specific feature importance for each of the three models - linear regression, random forest regressor, and gradient boosting regressor.
2. Use Permutation Importance to analyze feature importance statistically.

### Feature Importance from Linear Regression Model

In [ ]:
# feature importance from linear regression model
linear_model.fit(X_train, y_train)
linear_feature_importance = pd.Series(linear_model.coef_, index=X_train.columns).sort_values(ascending=False)
print("\nFeature Importance from Linear Regression Model:")
print(tabulate(linear_feature_importance.reset_index(), headers=['Feature', 'Importance'], tablefmt='grid'))

As can be seen from the output above, after the `squareMeters` feature, the next 3 most important features are

- `hasYard`
- `hasPool`
- `floors`

So, it looks like the cost of the property is also influenced by whether it has a yard or a pool, and the number of floors in the property. But looking at the coefficient value, the importance magnitude is much lower than that of the `squareMeters` feature.

After these, the `cityPartRange_10` seem to have the next highest importance, showing that this part of the city has a significant impact on the property prices, comparted to the other city parts.

### Feature Importance from Random Forest Regressor Model

In [ ]:
# feature importance for Random Forest Regressor
rf_model.fit(X_train, y_train)
rf_feature_importance = pd.Series(rf_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("\nFeature Importance from Random Forest Regressor:")
print(tabulate(rf_feature_importance.reset_index(), headers=['Feature', 'Importance'], tablefmt='grid'))

Apparently, while `squareMeters` is the most important feature, the next most important features are different in Random Forest Regressor model. `floors` is the only common top 5 feature between Linear Regression and Random Forest Regressor models.

### Feature Importance from Gradient Boosting Regressor Model

In [ ]:
# feature importance for gradient boosting regressor
gb_model.fit(X_train, y_train)
gb_feature_importance = pd.Series(gb_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("\nFeature Importance from Gradient Boosting Regressor:")
print(tabulate(gb_feature_importance.reset_index(), headers=['Feature', 'Importance'], tablefmt='grid'))

Putting aside the `squareMeters` feature, `hasYard` and `floors` seem to be the next most important top 5 features in the Gradient Boosting Regressor model that is in common with the Linear Regression model.

And `floors` is the only common top 5 feature between all three models, apart from the `squareMeters` feature.

### Feature Importance using Permutation Importance

In [ ]:
# Calculate permutation importance for linear regression model

linear_model.fit(X_train, y_train)

perm_importance = permutation_importance(
    linear_model, 
    X_test,  # Use test set for unbiased importance
    y_test, 
    n_repeats=10,  # Number of times to permute each feature
    random_state=42,
    scoring='neg_mean_squared_error'  # Use 'neg_mean_squared_error' for regression
)

# Get feature names (assuming X_train is a DataFrame)
feature_names = X_train.columns if hasattr(X_train, 'columns') else [f'feature_{i}' for i in range(X_train.shape[1])]

# Create a DataFrame with results
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance_mean': perm_importance.importances_mean, # type: ignore
    'importance_std': perm_importance.importances_std # type: ignore
})

# Sort by importance and get top 10
top_10_features = importance_df.sort_values('importance_mean', ascending=False).head(10)

print("Top 10 Features by Permutation Importance:")
print(top_10_features)

# Plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(range(10), top_10_features['importance_mean'][::-1])
plt.yticks(range(10), top_10_features['feature'][::-1]) # type: ignore
plt.xlabel('Permutation Importance')
plt.title('Top 10 Features by Permutation Importance')
plt.tight_layout()
plt.show()

Based on the Permutation Importance analysis, we can see that the `squareMeters` feature is by far the most important feature, followed by `floors`, `hasYard`, and `hasPool`.

Other than the primary feature `squareMeters`, the order of other features importance is different compared to the feature importance method used for linear regression.

In [ ]:
# Calculate permutation importance for random forest regressor model
rf_model.fit(X_train, y_train)  

perm_importance_rf = permutation_importance(
    rf_model, 
    X_test,  # Use test set for unbiased importance
    y_test, 
    n_repeats=10,  # Number of times to permute each feature
    random_state=42,
    scoring='neg_mean_squared_error'  # Use 'neg_mean_squared_error' for regression
)

# Get feature names (assuming X_train is a DataFrame)
feature_names_rf = X_train.columns if hasattr(X_train, 'columns') else [f'feature_{i}' for i in range(X_train.shape[1])]        

# Create a DataFrame with results
importance_df_rf = pd.DataFrame({
    'feature': feature_names_rf,
    'importance_mean': perm_importance_rf.importances_mean, # type: ignore
    'importance_std': perm_importance_rf.importances_std # type: ignore
})

# Sort by importance and get top 10
top_10_features_rf = importance_df_rf.sort_values('importance_mean', ascending=False).head(10)  

print("Top 10 Features by Permutation Importance (Random Forest):")
print(top_10_features_rf)

# Plot the results for Random Forest
plt.figure(figsize=(10, 6))
plt.barh(range(10), top_10_features_rf['importance_mean'][::-1])
plt.yticks(range(10), top_10_features_rf['feature'][::-1]) # type: ignore
plt.xlabel('Permutation Importance')
plt.title('Top 10 Features by Permutation Importance (Random Forest)')
plt.tight_layout()
plt.show()


The top 4 features identified by Permutation Importance are same as those identified for the Linear regression model. And these are

- `squareMeters`
- `floors`
- `hasYard`
- `hasPool`

In [ ]:
# Calculate permutation importance for gradient boosting regressor model
gb_model.fit(X_train, y_train)

perm_importance_gb = permutation_importance(
    gb_model, 
    X_test,  # Use test set for unbiased importance
    y_test, 
    n_repeats=10,  # Number of times to permute each feature
    random_state=42,
    scoring='neg_mean_squared_error'  # Use 'neg_mean_squared_error' for regression
)

# Get feature names (assuming X_train is a DataFrame)
feature_names_gb = X_train.columns if hasattr(X_train, 'columns') else [f'feature_{i}' for i in range(X_train.shape[1])]    

# Create a DataFrame with results
importance_df_gb = pd.DataFrame({
    'feature': feature_names_gb,
    'importance_mean': perm_importance_gb.importances_mean, # type: ignore
    'importance_std': perm_importance_gb.importances_std # type: ignore
})

# Sort by importance and get top 10
top_10_features_gb = importance_df_gb.sort_values('importance_mean', ascending=False).head(10)  
print("Top 10 Features by Permutation Importance (Gradient Boosting):")
print(top_10_features_gb)

# Plot the results for Gradient Boosting
plt.figure(figsize=(10, 6))
plt.barh(range(10), top_10_features_gb['importance_mean'][::-1])
plt.yticks(range(10), top_10_features_gb['feature'][::-1]) # type: ignore
plt.xlabel('Permutation Importance')
plt.title('Top 10 Features by Permutation Importance (Gradient Boosting)')
plt.tight_layout()
plt.show()

And with the gradient boosting regressor model, the top 3 are common between all three models:

- `squareMeters`
- `floors`
- `hasYard`

Putting all the results together, it appears that the size of the property in square meters along with the number of floors and whether it has a yard or a pool are the most important features that influence the property prices in this dataset.

The location of the property, represented by `cityPartRange`, also plays a significant role in determining the price, but it is not as significant as the size and features of the property itself.

### Feature Importance using SHAP Values

Let us now analyze the feature importance using SHAP values. SHAP values provide a unified measure of feature importance and can help us understand the contribution of each feature to the model's predictions.

#### SHAP Values for Linear Regression Model

In [ ]:
# SHAP explainer expects numerical data. Convert category columns to their codes.
X_train_shap = X_train.copy()
X_test_shap = X_test.copy()

for col in X_train_shap.select_dtypes(include='category').columns:
	X_train_shap[col] = X_train_shap[col].cat.codes
	X_test_shap[col] = X_test_shap[col].cat.codes

# Convert boolean columns to integers
for col in X_train_shap.select_dtypes(include='bool').columns:
	X_train_shap[col] = X_train_shap[col].astype(int)
	X_test_shap[col] = X_test_shap[col].astype(int)

# Create SHAP explainer
explainer = shap.Explainer(linear_model, X_train_shap)

# Calculate SHAP values
shap_values = explainer(X_test_shap)

# Plot SHAP values
shap.summary_plot(shap_values, X_test_shap)

The SHAP values order seem to be same for the first 5 features as the feature importance from the Linear Regression model.

In [ ]:
# SHAP dependence plot for squareMeters
shap.dependence_plot('squareMeters', shap_values.values, X_test_shap,
                     interaction_index=None, show=False)
plt.title('SHAP Dependence Plot for squareMeters')
plt.show()

In [ ]:
# SHAP Dependence plot for hasYard
shap.dependence_plot('hasYard', shap_values.values, X_test_shap,
                     interaction_index=None, show=False)
plt.title('SHAP Dependence Plot for hasYard')
plt.show()

In [ ]:
# SHAP Dependence plot for floors
shap.dependence_plot('floors', shap_values.values, X_test_shap,
                     interaction_index=None, show=False)
plt.title('SHAP Dependence Plot for floors')
plt.show()

**Analysis of SHAP Dependence Plots:**

- Strong positive impact of `squareMeters` on price, as expected
- `floors` too shows a clear positive relationship with price, indicating that more floors generally lead to higher prices, but the strength is lower
- `hasYard` shows a clear positive relationship with price, indicating that properties with yards tend to have higher prices

We only analyzed three of top 5 features, in order to compare with the SHAP values from Random Forest Regressor model.

Since `hasPool` is also binary in nature, we omitted it from dependency plot analysis, as it would show similar results to `hasYard`.

#### SHAP Values for Random Forest Regressor Model

In [ ]:
# SHAP with Random Forest Regressor
explainer_rf = shap.Explainer(rf_model, X_train_shap)
shap_values_rf = explainer_rf(X_test_shap)

# Plot SHAP values for Random Forest Regressor
shap.summary_plot(shap_values_rf, X_test_shap)

In [ ]:
# SHAP dependence plot for squareMeters
shap.dependence_plot('squareMeters', shap_values_rf.values, X_test_shap,
                     interaction_index=None, show=False)
plt.title('SHAP Dependence Plot for squareMeters')
plt.show()

In [ ]:
# SHAP dependence plot for floors
shap.dependence_plot('floors', shap_values_rf.values, X_test_shap,
                     interaction_index=None, show=False)
plt.title('SHAP Dependence Plot for floors')
plt.show()

In [ ]:
# SHAP dependence for hasYard
shap.dependence_plot('hasYard', shap_values_rf.values, X_test_shap,
                     interaction_index=None, show=False)
plt.title('SHAP Dependence Plot for hasYard')
plt.show()

**Analysis of SHAP Dependence Plots:**

- Strong positive impact of `squareMeters` on price, as expected
- The impact of `floors` is also positive, but since the Random Forest is a non-linear model, the relationship is not as clear as in the Linear Regression model.
- And `hasYard` while showing a postive relationship, it does have significant noise in the SHAP values, indicating that the model is not able to explain the relationship between `hasYard` and `price` as well as it does for `squareMeters` and `floors`.

### SHAP Values for Gradient Boosting Regressor Model

In [ ]:
# SHAP values for Gradient Boosting Regressor Model
explainer_gb = shap.Explainer(gb_model, X_train_shap)
shap_values_gb = explainer_gb(X_test_shap)  

# SHAP values for Gradient Boosting Regressor Model
shap.summary_plot(shap_values_gb, X_test_shap)  

In [ ]:
# shap dependence plot for squareMeters
shap.dependence_plot('squareMeters', shap_values_gb.values, X_test_shap,
                     interaction_index=None, show=False)
plt.title('SHAP Dependence Plot for squareMeters')
plt.show()

In [ ]:
# SHAP dependence plot for floors
shap.dependence_plot('floors', shap_values_gb.values, X_test_shap,
                     interaction_index=None, show=False)
plt.title('SHAP Dependence Plot for floors')
plt.show()

In [ ]:
# SHAP dependence plot for hasYard
shap.dependence_plot('hasYard', shap_values_gb.values, X_test_shap,
                     interaction_index=None, show=False)
plt.title('SHAP Dependence Plot for hasYard')
plt.show()

**Analysis of SHAP Dependence Plots:**

- Strong positive impact of `squareMeters` on price, as expected
- `floors` has a positive impact, but less pronounced than `squareMeters`. There is some kind of steps noticed in the SHAP values, indicating that if the number of floors is higher than the mean floors, the price increases significantly.
- `hasYard` has a slightly better relationship with price with Gradient Boosting Regressor model compared to Random Forest model, as noticed from lesser overlap of the SHAP values.

## Final Model Selection

We shall use the Linear Regression model as the final model for this task, as it is the simplest model that is able to explain 100% of the variance in the target variable `price`.

As we saw earlier, top 5 features of the linear regression model are more than sufficient to explain the variance in the target variable `price`, and the model is able to predict the price of the properties with a very high accuracy.

The top 5 features are:
- `squareMeters`
- `floors`
- `hasYard`
- `hasPool`
- `cityPartRange_10`

In [ ]:
# select the top 5 features from the original dataset
top_5_features = ['squareMeters', 'floors', 'hasYard', 'hasPool', 'cityPartRange']

paris_real_estate_top5 = paris_real_estate_processing[top_5_features + ['price']]

# split the top 5 features dataset into train and test sets
X_train_top5, X_test_top5, y_train_top5, y_test_top5 = train_test_split(
    paris_real_estate_top5.drop(columns=['price']),
    paris_real_estate_top5['price'],
    test_size=0.2,
    random_state=42
)

# standardize the features that need to be standardized
scaler_top5 = StandardScaler()
X_train_top5[['squareMeters', 'floors']] = scaler_top5.fit_transform(X_train_top5[['squareMeters', 'floors']])
X_test_top5[['squareMeters', 'floors']] = scaler_top5.transform(X_test_top5[['squareMeters', 'floors']])

# transform cityPartRange to 0 or 1 based on the if the cityPartRange is 10 or not
X_train_top5['cityPartRange_10'] = (X_train_top5['cityPartRange'] == 10).astype(int)
X_test_top5['cityPartRange_10'] = (X_test_top5['cityPartRange'] == 10).astype(int)

# drop the original cityPartRange column
X_train_top5 = X_train_top5.drop(columns=['cityPartRange'])
X_test_top5 = X_test_top5.drop(columns=['cityPartRange'])

# print the shape of the top 5 features train and test sets
print(f"\nShape of the training set with top 5 features: {X_train_top5.shape}")
print(f"Shape of the testing set with top 5 features: {X_test_top5.shape}")

# save the processed top 5 features train and test sets to CSV files
X_train_top5.to_csv('X_train_top5_processed.csv', index=False)
X_test_top5.to_csv('X_test_top5_processed.csv', index=False)
y_train_top5.to_csv('y_train_top5_processed.csv', index=False)
y_test_top5.to_csv('y_test_top5_processed.csv', index=False)

In [ ]:
# train the linear regression model on the entire training set
linear_model_top5 = LinearRegression()
linear_model_top5.fit(X_train_top5, y_train_top5)

# print the coefficients of the linear regression model
linear_coefficients = pd.Series(linear_model_top5.coef_, index=X_train_top5.columns).sort_values(ascending=False)
print("\nLinear Regression Model Coefficients for Top 5 Features:")
print(tabulate(linear_coefficients.reset_index(), headers=['Feature', 'Coefficient'], tablefmt='grid'))

In [ ]:
# print the coefficients of original linear regression model for top 5 features
original_linear_coefficients = pd.Series(linear_model.coef_, index=X_train.columns).sort_values(ascending=False)
print("\nOriginal Linear Regression Model Coefficients for Top 5 Features:")
print(tabulate(original_linear_coefficients.head(5).reset_index(), headers=['Feature', 'Coefficient'], tablefmt='grid'))

The coefficients are pretty similar between the original linear regression model and the model trained on the top 5 features, indicating that the top 5 features are sufficient to explain the variance in the target variable `price`.

In [ ]:
# evaluation metrics for the top5 linear regression model
y_pred_top5 = linear_model_top5.predict(X_test_top5)
mse_top5 = mean_squared_error(y_test_top5, y_pred_top5)
rmse_top5 = mse_top5 ** 0.5
r2_top5 = r2_score(y_test_top5, y_pred_top5)  

# print the evaluation metrics
print(f"\nEvaluation Metrics for Top 5 Linear Regression Model:")
print(f"Mean Squared Error (MSE): {mse_top5:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse_top5:.2f}")
print(f"R-squared (R^2): {r2_top5:.2f}")        

Let us now create a wrapper function that will take just one input and predict the price of the property.

In [ ]:
# function to predict the price of a property based on top 5 features
def predict_price_top5(square_meters, floors, has_yard, has_pool, city_part_range):
    """
    Predict the price of a property based on top 5 features.
    
    Parameters:
    square_meters (float): Size of the property in square meters.
    floors (int): Number of floors in the property.
    has_yard (bool): Whether the property has a yard or not.
    has_pool (bool): Whether the property has a pool or not.
    city_part_range (int): City part range (e.g., 10).
    
    Returns:
    float: Predicted price of the property.
    """
    # create a DataFrame with the input features
    input_data = pd.DataFrame({
        'squareMeters': [square_meters],
        'floors': [floors],
        'hasYard': [1 if has_yard else 0],
        'hasPool': [1 if has_pool else 0],
        'cityPartRange_10': [1 if city_part_range == 10 else 0]
    })
    
    # standardize the features
    input_data[['squareMeters', 'floors']] = scaler_top5.transform(input_data[['squareMeters', 'floors']])
    
    # predict the price
    predicted_price = linear_model_top5.predict(input_data)
    
    return predicted_price[0]

In [ ]:
# sample input for prediction
# this is very close to the second sample in the dataset
sample_square_meters = 80000
sample_floors = 90
sample_has_yard = True
sample_has_pool = True
sample_city_part_range = 8

In [ ]:
# predict the price of the property
predicted_price = predict_price_top5(sample_square_meters, sample_floors, sample_has_yard, sample_has_pool, sample_city_part_range)
print(f"\nPredicted price for the property is: {predicted_price:.2f}")

In [ ]:
# actual price for the second sample in the dataset
actual_price = paris_real_estate_top5.iloc[1]['price']
print(f"\nActual price for the property is: {actual_price:.2f}")

The output predicted based on just 5 top features with a linear regression model is very close to the actual price for the second sample in the dataset, indicating that the model is able to predict the price of the property with a very high accuracy.

We shall need the model and the scaler to predict the price of the property. We will save both the model and the scaler to files so that we can use them later for prediction without needing to retrain the model.

In [ ]:
# save the model to a file
joblib.dump(linear_model_top5, 'linear_model_top5.pkl')
print("\nLinear Regression Model for Top 5 Features saved to 'linear_model_top5.pkl'.")

# save the scaler to a file
joblib.dump(scaler_top5, 'scaler_top5.pkl')
print("Scaler for Top 5 Features saved to 'scaler_top5.pkl'.")